In [1]:
import yaml
from pathlib import Path

import numpy as np
import torch
import torchvision
from torch import nn
import torch.nn.functional as F
import sys

sys.path.append(str(Path().absolute().parent))
from models import nets
Model = nets.ResNet3D_18_Classifier

In [2]:
model = Model(True, 3, 1)

Freezing layers
Initializing network for 3 channel input


In [3]:
type(model.parameters())

generator

In [4]:
low_lr_list = []
high_lr_list = []
for name,param in model.named_parameters():
    if 'fc' not in name:
        low_lr_list.append(param)
    else:
        high_lr_list.append(param)

In [5]:
model[0].fc.bias

Parameter containing:
tensor([-0.0284], requires_grad=True)

In [6]:
model[0]

VideoResNet(
  (stem): BasicStem(
    (0): Conv3d(3, 64, kernel_size=(3, 7, 7), stride=(1, 2, 2), padding=(1, 3, 3), bias=False)
    (1): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
  )
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Sequential(
        (0): Conv3DSimple(64, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=False)
        (1): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU(inplace=True)
      )
      (conv2): Sequential(
        (0): Conv3DSimple(64, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=False)
        (1): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (relu): ReLU(inplace=True)
    )
    (1): BasicBlock(
      (conv1): Sequential(
        (0): Conv3DSimple(64, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=False)
        (1):

In [7]:
from torch import optim

In [8]:
x = 0.1
optimizer = optim.Adam([
            {'params': low_lr_list, 'lr':x/10000},
            {'params': high_lr_list, 'lr': x}
        ], lr=x)

In [9]:
optimizer

Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    lr: 1e-05
    weight_decay: 0

Parameter Group 1
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    lr: 0.1
    weight_decay: 0
)

In [35]:
model = torch.hub.load('pytorch/vision:v0.6.0', 'squeezenet1_0', pretrained=True)

Using cache found in /home/linardos/.cache/torch/hub/pytorch_vision_v0.6.0


In [36]:
model.classifier[1]=nn.Conv2d(512, 1, kernel_size=(1, 1), stride=(1, 1))

In [37]:
model.classifier

Sequential(
  (0): Dropout(p=0.5, inplace=False)
  (1): Conv2d(512, 1, kernel_size=(1, 1), stride=(1, 1))
  (2): ReLU(inplace=True)
  (3): AdaptiveAvgPool2d(output_size=(1, 1))
)

## Reduce channel number but get the weights corresponding to one of the channels:

In [46]:
trained_kernel = model.features[0].weight
model.features[0]=nn.Conv2d(1, 96, kernel_size=(7, 7), stride=(2, 2))
model.features[0].weight = torch.nn.Parameter(trained_kernel[:,0,...].unsqueeze(1)) 

In [47]:
model.features[0].weight.shape

torch.Size([96, 7, 7])

In [45]:
model.features[0].weight.shape

torch.Size([96, 1, 7, 7])